<a href="https://colab.research.google.com/github/MichalRyszardWojcik/hello-world/blob/master/2020_08_27_Fibonacci.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import numpy as np
! pip install -q -U trax
import trax

     |████████████████████████████████| 368kB 3.4MB/s 
     |████████████████████████████████| 2.6MB 10.9MB/s 
     |████████████████████████████████| 163kB 35.2MB/s 
     |████████████████████████████████| 1.5MB 35.7MB/s 
     |████████████████████████████████| 3.5MB 46.1MB/s 
     |████████████████████████████████| 51kB 5.4MB/s 
     |████████████████████████████████| 778kB 42.7MB/s 
     |████████████████████████████████| 307kB 44.6MB/s 
     |████████████████████████████████| 1.1MB 45.6MB/s 
     |████████████████████████████████| 368kB 41.3MB/s 
     |████████████████████████████████| 5.3MB 39.2MB/s 
     |████████████████████████████████| 184kB 49.1MB/s 
     |████████████████████████████████| 358kB 48.1MB/s 
     |████████████████████████████████| 81kB 6.9MB/s 
     |████████████████████████████████| 655kB 48.8MB/s 
     |████████████████████████████████| 983kB 44.4MB/s 
     |████████████████████████████████| 890kB 34.0MB/s 
     |████████████████████████████████| 3.0MB 41.4MB/

In [2]:
def Fibo(a1,a2,length):
  #if length == 0: return []
  #if length == 1: return [a1]
  if length == 2: return [a1,a2]
  a3 = a1+a2
  return [a1] + Fibo(a2,a3,length-1)

In [3]:
def RandomFibos(batch_size, length, start):
  seeds = np.random.randint(1, start, (batch_size,2))
  Fibos = []
  for i in range(batch_size):
    a1 = seeds[i][0]
    a2 = seeds[i][1]
    if (a1 > a2): b = a1; a1 = a2; a2 = b
    Fibos += Fibo(a1,a2,length)
  return np.array(Fibos).reshape(batch_size,length)

g_Fibolength = length of sequences generated as training input

g_Fibostart = the maximum integer used in seeding random Fibonacci sequences (they are determined by the first two terms, this is the maximum second term)

g_Fibolength = 10 and g_Fibostart = 74 are the maximum numbers so that vocab_size is slightly below 64x64 = 4096 - the number we need for the chess project

Unfortunately, it is unrealistic to use these numbers because the training is either absurdly slow or absurdly inaccurate.


In [10]:
g_Fibolength = 7 #10
g_Fibostart = 11 #74

maxFibo = Fibo(g_Fibostart,g_Fibostart,g_Fibolength)
print(f'maxFibo={maxFibo}')
g_vocab_size = np.max(maxFibo) + 1
print(f'vocab_size={g_vocab_size}')
print(f'64x64={64*64}')

def Fibo_transformer_lm(mode='train'):
  return trax.models.TransformerLM(  
          d_model=32,
          d_ff=128,
          n_layers=2, 
          vocab_size= g_vocab_size,
          mode=mode)

maxFibo=[11, 11, 22, 33, 55, 88, 143]
vocab_size=144
64x64=4096


In [11]:
def training_input(batch_size, length):
  while True:
    Fibos = RandomFibos(batch_size=batch_size, length=g_Fibolength, start=g_Fibostart)
    inputs = Fibos
    targets = Fibos
    loss_weights = np.ones((batch_size, length), np.int32)
    yield (inputs,targets,loss_weights)

trax_inputs = trax.data.inputs.Inputs(lambda _: training_input(batch_size=16, length=g_Fibolength))

In [12]:
next(training_input(7,g_Fibolength))

(array([[  8,   8,  16,  24,  40,  64, 104],
        [  2,   7,   9,  16,  25,  41,  66],
        [  1,   1,   2,   3,   5,   8,  13],
        [  4,   7,  11,  18,  29,  47,  76],
        [  9,  10,  19,  29,  48,  77, 125],
        [  4,   5,   9,  14,  23,  37,  60],
        [  5,   9,  14,  23,  37,  60,  97]]),
 array([[  8,   8,  16,  24,  40,  64, 104],
        [  2,   7,   9,  16,  25,  41,  66],
        [  1,   1,   2,   3,   5,   8,  13],
        [  4,   7,  11,  18,  29,  47,  76],
        [  9,  10,  19,  29,  48,  77, 125],
        [  4,   5,   9,  14,  23,  37,  60],
        [  5,   9,  14,  23,  37,  60,  97]]),
 array([[1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1]], dtype=int32))

In [13]:
output_dir = os.path.expanduser('~/train_dir/')
!rm -f ~/train_dir/model.pkl.gz  # Remove old model.

# Train tiny model with Trainer.
trainer = trax.supervised.Trainer(
    model=Fibo_transformer_lm,
    loss_fn=trax.layers.CrossEntropyLoss(),
    optimizer=trax.optimizers.Adafactor,  # Change optimizer params here.
    lr_schedule=trax.lr.constant(0.001),  # Change lr schedule here.
    inputs=trax_inputs,
    output_dir=output_dir)

n_epochs  = 5
train_steps = 500
eval_steps = 2
for _ in range(n_epochs):
  trainer.train_epoch(train_steps, eval_steps)



Step    500: Ran 500 train steps in 66.62 secs
Step    500: Evaluation
Step    500: train                   accuracy |  0.45535713
Step    500: train                       loss |  2.53289914
Step    500: train         neg_log_perplexity | -2.53289914
Step    500: train          sequence_accuracy |  0.00000000
Step    500: train weights_per_batch_per_core |  112.00000000
Step    500: eval                    accuracy |  0.45089287
Step    500: eval                        loss |  2.52572608
Step    500: eval          neg_log_perplexity | -2.52572608
Step    500: eval           sequence_accuracy |  0.00000000
Step    500: eval  weights_per_batch_per_core |  112.00000000
Step    500: Finished evaluation

Step   1000: Ran 500 train steps in 4.43 secs
Step   1000: Evaluation
Step   1000: train                   accuracy |  0.69196427
Step   1000: train                       loss |  1.04891539
Step   1000: train         neg_log_perplexity | -1.04891539
Step   1000: train          sequence_acc

In [14]:
def output(input):
  predict_model = Fibo_transformer_lm(mode='predict')
  predict_signature = trax.shapes.ShapeDtype((1,1), dtype=np.int32)
  predict_model.init_from_file(os.path.join(output_dir, "model.pkl.gz"),weights_only=True, input_signature=predict_signature)
  return trax.supervised.decoding.autoregressive_sample(predict_model, input, temperature=0.0, max_length=g_Fibolength+3, eos_id = -1)

In [15]:
input = np.array([[1,1]])
print(output(input))
input = np.array([[2,2]])
print(output(input))


[[ 3  5  8 13 21 34 55 89  3 89]]
[[ 5  7 12 19 31 50 81  5  8 13]]


In [16]:
input = np.array([[3]])
output(input)

array([[ 9, 12, 21, 33, 54, 87, 19, 29, 48, 77]])

In [17]:
input = np.array([[3,3]])
output(input)

array([[ 5,  8, 13, 21, 34, 55, 89, 28, 46, 74]])